In [45]:
import os
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from svea_expvis.config import Config
from svea_expvis.readers.txt import text_reader
from svea_expvis.bokeh_widgets.map import get_map
from svea_expvis.bokeh_widgets.callbacks import (
    lasso_corr_callback, 
    select_callback, 
    month_selection_callback,
    update_colormapper, 
    range_selection_callback, 
    range_slider_update_callback, 
)
from svea_expvis.bokeh_widgets.utils import get_columndata_source

from bokeh.models import LinearColorMapper, ColorBar
from bokeh.transform import transform
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, PreText, Select, Circle, RangeSlider
from bokeh.layouts import column, row


DEFAULT_PARAMETERS = ['TEMP_BTL', 'TEMP_CTD', 'SALT_BTL', 'SALT_CTD', 'DOXY_BTL', 'DOXY_CTD', 'FLUO_CTD', 'CPHL']

cfig = Config(data_file_name='phyche_archive_2020.feather')

output_notebook()


Loading BokehJS ...

In [46]:
df = pd.read_feather(cfig.data_path)
df['MONTH'] = df['timestamp'].dt.month

position_master_source = get_columndata_source(
    df.loc[:, ['STATN', 'LATIT', 'LONGI', 'KEY', 'MONTH']].drop_duplicates().reset_index(drop=True), 
    lat_col='LATIT', lon_col='LONGI',
)
position_source = get_columndata_source(
    df.loc[:, ['STATN', 'LATIT', 'LONGI', 'KEY', 'MONTH']].drop_duplicates().reset_index(drop=True), 
    lat_col='LATIT', lon_col='LONGI',
)

data_source = get_columndata_source(df.loc[:, ['KEY', 'DEPH'] + DEFAULT_PARAMETERS], 'FLUO_CTD', 'CPHL')

In [47]:
callback_month = month_selection_callback(position_source=position_source, position_master_source=position_master_source)
month_selector = Select(
    title="Select month",
    value='All',
    options=['All'] + pd.date_range(start='2020-01', freq='M', periods=12).month_name().to_list(),
)
month_selector.js_on_change('value', callback_month)
callback_month.args["month"] = month_selector

In [48]:
fig_map = get_map()

map_renderer = fig_map.circle(
    x="LONGI", 
    y="LATIT",
    size=10, alpha=0.5, source=position_source,
)

nonselected_circle = Circle(fill_alpha=0.3, line_color='grey')
map_renderer.nonselection_glyph = nonselected_circle


In [49]:
TOOLTIPS = [
    ("Serie", "@key"),
    ("Depth", "@dep")]
corr = figure(
    width=400, height=400,
    title='Correlation of selected X- and Y values',
    tools='pan,wheel_zoom,box_select,reset,save',
    active_scroll="wheel_zoom",
    tooltips=TOOLTIPS
)
corr.xaxis.axis_label = 'FLUO_CTD'
corr.yaxis.axis_label = 'CPHL'
corr_source = ColumnDataSource(data=dict(x=[], y=[], dep=[], key=[]))
corr_renderer = corr.circle(
    x="x", 
    y="y",
    size=3, color='red', alpha=0.5, source=corr_source,
)
nonselected_corr_circle = Circle(fill_alpha=0.05, line_color='grey')
corr_renderer.nonselection_glyph = nonselected_corr_circle

In [50]:
x_prof = figure(
    width=400, height=400,
    title='Selected X-profiles',
    tools='pan,wheel_zoom,box_select,reset,save',
    active_scroll="wheel_zoom",
)
x_prof.xaxis.axis_label = 'FLUO_CTD'
x_prof.yaxis.axis_label = 'Depth (m)'
x_prof.y_range.flipped = True
x_prof.circle(
    x="x", 
    y="dep",
    size=3, alpha=0.5, source=corr_source,
)

GlyphRenderer(id='3076', ...)

In [51]:
y_prof = figure(
    width=400, height=400,
    title='Selected Y-profiles',
    tools='pan,wheel_zoom,box_select,reset,save',
    active_scroll="wheel_zoom",
)
y_prof.xaxis.axis_label = 'CPHL'
y_prof.yaxis.axis_label = 'Depth (m)'
y_prof.y_range.flipped = True
y_prof.circle(
    x="y", 
    y="dep",
    size=3, alpha=0.5, source=corr_source,
)

GlyphRenderer(id='3114', ...)

In [52]:
x_sel = Select(value='FLUO_CTD', options=DEFAULT_PARAMETERS, title='X-parameter')
y_sel = Select(value='CPHL', options=DEFAULT_PARAMETERS, title='Y-parameter')

x_sel.js_on_change("value", select_callback(data_source=data_source, axis_objs=[corr.xaxis[0], x_prof.xaxis[0]], axis='x'))
y_sel.js_on_change("value", select_callback(data_source=data_source, axis_objs=[corr.yaxis[0], y_prof.xaxis[0]], axis='y'))  # (y_prof.xaxis[0] is correct, y and x: yes, I know :), you sure?!?! YES! :D)

In [53]:
depth_slider = RangeSlider(start=0, end=100, value=(0, 100),
                           step=0.5, title="Select depth range", 
                           width=300)
depth_slider.js_on_change('value', range_selection_callback(data_source=corr_source))

In [54]:
selection_lasso_change = lasso_corr_callback(
     x_selector=x_sel,
     y_selector=y_sel,
     data_source=data_source,
     position_source=position_source,
     corr_source=corr_source,
     corr_plot=corr,
)

position_source.selected.js_on_change(
    'indices',
    selection_lasso_change,
    range_slider_update_callback(slider=depth_slider, data_source=corr_source),
)

In [55]:
show(row(column(fig_map, row(x_sel, month_selector), row(y_sel, depth_slider)), column(row(x_prof, y_prof), corr)))